# DeepLabCut Toolbox - Colab
https://github.com/AlexEMG/DeepLabCut

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

###This notebook assumes you already have a project folder with labeled data! 

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"->select "Python3", and then select "GPU"


In [1]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut

     |████████████████████████████████| 696kB 14.2MB/s 
     |████████████████████████████████| 17.3MB 200kB/s 
     |████████████████████████████████| 112kB 58.3MB/s 
     |████████████████████████████████| 9.5MB 44.4MB/s 
     |████████████████████████████████| 296kB 59.9MB/s 
     |████████████████████████████████| 184kB 44.2MB/s 
     |████████████████████████████████| 3.1MB 45.7MB/s 
     |████████████████████████████████| 13.1MB 250kB/s 
     |████████████████████████████████| 37.6MB 155kB/s 
     |████████████████████████████████| 552kB 46.5MB/s 
  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=bc0fae04f69390e59902c1d64530502729e08e7fa06d08db2141ad27a86f4902
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
  Created wheel for filterpy: filename=filterpy-1.4.5-cp37-none-any.whl size=110452 sha256=93ee5c3bc98389feac88c650dbe2e93597cf6aca31fcc4399bb17016cd381

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [1]:
# Use TensorFlow 1.13.1:
%tensorflow_version 1.13.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.1`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [3]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [4]:
#Setup your project variables:
# PLEASE EDIT THESE:
  
ProjectFolderName = 'icteste2/icteste-triangulo-red-2021-03-09'
VideoType = 'mp4' 

#don't edit these:
videofile_path = ['/content/drive/My Drive/'+ProjectFolderName+'/videos/'] #Enter the list of videos or folder to analyze.
videofile_path

['/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos/']

In [6]:
#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"

In [7]:
import deeplabcut

DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [8]:
deeplabcut.__version__

'2.1.10.2'

In [9]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before: 
path_config_file = '/content/drive/My Drive/'+ProjectFolderName+'/config.yaml'
path_config_file

'/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/config.yaml'

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [15]:
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models! 
#Then, run this cell. There are many more functions you can set here, including which netowkr to use!
#check the docstring for full options you can do!
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/training-datasets/iteration-0/UnaugmentedDataSet_ictestemar9  already exists!
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/dlc-models/iteration-0/ictestemar9-trainset95shuffle1  already exists!
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/dlc-models/iteration-0/ictestemar9-trainset95shuffle1/train  already exists!
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/dlc-models/iteration-0/ictestemar9-trainset95shuffle1/test  already exists!
The training dataset is successfully created. Use the function 'train_network' to start training. Happy training!


[(0.95,
  1,
  (array([ 5, 16,  2, 10,  3, 15, 13,  4,  1, 19, 18,  6,  8, 17, 11,  7, 14,
           9, 12]), array([0])))]

## Start training:
This function trains the network for a specific shuffle of the training dataset. 

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU... 
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations). 
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

Config:
{'all_joints': [[0]],
 'all_joints_names': ['pontacaneta'],
 'alpha_r': 0.02,
 'batch_size': 1,
 'crop_pad': 0,
 'cropratio': 0.4,
 'dataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ictestemar9/icteste_triangulo-red95shuffle1.mat',
 'dataset_type': 'imgaug',
 'decay_steps': 30000,
 'deterministic': False,
 'display_iters': 1000,
 'fg_fraction': 0.25,
 'global_scale': 0.8,
 'init_weights': '/usr/local/lib/python3.7/dist-packages/deeplabcut/pose_estimation_tensorflow/models/pretrained/resnet_v1_50.ckpt',
 'intermediate_supervision': False,
 'intermediate_supervision_layer': 12,
 'location_refinement': True,
 'locref_huber_loss': True,
 'locref_loss_weight': 0.05,
 'locref_stdev': 7.2801,
 'log_dir': 'log',
 'lr_init': 0.0005,
 'max_input_size': 1500,
 'mean_pixel': [123.68, 116.779, 103.939],
 'metadataset': 'training-datasets/iteration-0/UnaugmentedDataSet_ictestemar9/Documentation_data-icteste_95shuffle1.pickle',
 'min_input_size': 64,
 'mirror': False,
 'multi_step'

Selecting single-animal trainer
Starting with imgaug pose-dataset loader (=default).
Batch Size is 1
Initializing ResNet
Loading ImageNet-pretrained resnet_50
Display_iters overwritten as 10
Save_iters overwritten as 500
Training parameter:
{'stride': 8.0, 'weigh_part_predictions': False, 'weigh_negatives': False, 'fg_fraction': 0.25, 'mean_pixel': [123.68, 116.779, 103.939], 'shuffle': True, 'snapshot_prefix': '/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/dlc-models/iteration-0/ictestemar9-trainset95shuffle1/train/snapshot', 'log_dir': 'log', 'global_scale': 0.8, 'location_refinement': True, 'locref_stdev': 7.2801, 'locref_loss_weight': 0.05, 'locref_huber_loss': True, 'optimizer': 'sgd', 'intermediate_supervision': False, 'intermediate_supervision_layer': 12, 'regularize': False, 'weight_decay': 0.0001, 'crop_pad': 0, 'scoremap_dir': 'test', 'batch_size': 1, 'dataset_type': 'imgaug', 'deterministic': False, 'mirror': False, 'pairwise_huber_loss': False, 'weigh_on

A saída de streaming foi truncada nas últimas 5000 linhas.
iteration: 170160 loss: 0.0016 lr: 0.02
iteration: 170170 loss: 0.0013 lr: 0.02
iteration: 170180 loss: 0.0039 lr: 0.02
iteration: 170190 loss: 0.0017 lr: 0.02
iteration: 170200 loss: 0.0017 lr: 0.02
iteration: 170210 loss: 0.0017 lr: 0.02
iteration: 170220 loss: 0.0014 lr: 0.02
iteration: 170230 loss: 0.0019 lr: 0.02
iteration: 170240 loss: 0.0016 lr: 0.02
iteration: 170250 loss: 0.0021 lr: 0.02
iteration: 170260 loss: 0.0019 lr: 0.02
iteration: 170270 loss: 0.0018 lr: 0.02
iteration: 170280 loss: 0.0016 lr: 0.02
iteration: 170290 loss: 0.0018 lr: 0.02
iteration: 170300 loss: 0.0026 lr: 0.02
iteration: 170310 loss: 0.0019 lr: 0.02
iteration: 170320 loss: 0.0015 lr: 0.02
iteration: 170330 loss: 0.0019 lr: 0.02
iteration: 170340 loss: 0.0011 lr: 0.02
iteration: 170350 loss: 0.0015 lr: 0.02
iteration: 170360 loss: 0.0020 lr: 0.02
iteration: 170370 loss: 0.0020 lr: 0.02
iteration: 170380 loss: 0.0012 lr: 0.02
iteration: 170390 los

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [10]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler, 
#so be sure your labels are good! (And you have trained enough ;)

/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/evaluation-results/  already exists!
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/evaluation-results/iteration-0/ictestemar9-trainset95shuffle1  already exists!
Running  DLC_resnet_50_ictestemar9shuffle1_385000  with # of trainingiterations: 385000
This net has already been evaluated!


  0%|          | 0/20 [00:00<?, ?it/s]

Plotting...(attention scale might be inconsistent in comparison to when data was analyzed; i.e. if you used rescale)
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/evaluation-results/iteration-0/ictestemar9-trainset95shuffle1/LabeledImages_DLC_resnet_50_ictestemar9shuffle1_385000_snapshot-385000  already exists!


<IPython.core.display.Javascript object>

  5%|▌         | 1/20 [00:01<00:28,  1.50s/it]

...

 90%|█████████ | 18/20 [00:20<00:02,  1.10s/it]

<IPython.core.display.Javascript object>

 95%|█████████▌| 19/20 [00:22<00:01,  1.15s/it]

<IPython.core.display.Javascript object>

100%|██████████| 20/20 [00:23<00:00,  1.16s/it]


## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos... 
- pplease see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos: 
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [11]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

Using snapshot-385000 for model /content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/dlc-models/iteration-0/ictestemar9-trainset95shuffle1
Initializing ResNet
Analyzing all the videos in the directory...
Starting to analyze %  /content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos/triangulo-red.mp4
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos  already exists!
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet_50_ictestemar9shuffle1_385000'

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [12]:
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType)

Analyzing all the videos in the directory...
Loading  /content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos/triangulo-red.mp4 and data.
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos/plot-poses/triangulo-red  already exists!


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Plots created! Please check the directory "plot-poses" within the video directory


Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4. 

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [13]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)

Analyzing all the videos in the directory...
/content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos  already exists!
Starting to process video: /content/drive/My Drive/icteste2/icteste-triangulo-red-2021-03-09/videos/triangulo-red.mp4
Labeled video triangulo-red already created.
